In [ ]:
import os
from typing import Any
import nltk
import nltk.internals
import pandas as pd

In [ ]:
from pydantic import BaseModel
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate

##### subtheme

In [ ]:
groups = [
{'Name': 'Seeking clarity and reassurance through multiple medical opinions', 'Description': "Parents actively pursue multiple medical opinions to gain reassurance and clarity about their child's condition. This proactive approach reflects their need for confidence in treatment decisions and understanding of the diagnosis. They often feel overwhelmed by the complexity of medical information and seek validation from various healthcare professionals to alleviate their anxiety and uncertainty.", 'Codes': ['Seeking multiple opinions for reassurance and clarity', 'Desire for positive medical opinions and reassurance', 'Frustration with lack of data and statistics', 'Desire for comprehensive data on surgery outcomes for AAOCA', 'Desire for advancements in surgical techniques and technology', 'Need for clear communication and transparency from healthcare providers']},
{'Name': 'Coping with emotional turmoil and uncertainty surrounding the diagnosis', 'Description': "Parents experience significant emotional distress upon receiving their child's diagnosis and during treatment. They articulate feelings of devastation, fear, and anxiety, grappling with the uncertainty of their child's health. This emotional turmoil is compounded by the need for support and the pressure to make informed decisions in a high-stakes environment.", 'Codes': ['Describing the emotional impact of diagnosis and treatment', 'Emotional turmoil over surgical decisions', 'The emotional toll of waiting for surgery outcomes', 'Overwhelming feelings of uncertainty and fear for the future', "Anxiety and worry about child's health and future", 'The emotional turmoil of receiving a life-threatening diagnosis']},
{'Name': 'Balancing the desire for normalcy with health concerns', 'Description': "Parents express a strong desire for their children to lead active lives and participate in normal activities despite health risks. They struggle with the balance between encouraging their child's passions and protecting them from potential dangers. This reflects their hope for normalcy in their child's life while managing the complexities of their medical condition.", 'Codes': ['Desire for active lifestyle despite health concerns', "Normalizing the child's experience post-surgery", "Impact of sports restrictions on children's lives and identities", 'The desire for normalcy and active family life']},
{'Name': 'Experiencing frustration and isolation in the healthcare system', 'Description': "Parents express frustration over the lack of awareness and understanding of AAOCA among healthcare professionals. They feel isolated in their experiences, often struggling to find clear information and support. This frustration is compounded by the emotional burden of navigating a complex medical system while advocating for their child's health.", 'Codes': ['Frustration with the lack of awareness among medical professionals', 'Struggles with communication about the condition', 'Challenges in discussing the condition with family and friends', 'The struggle for information and understanding from medical professionals', 'Frustration over missed early diagnosis of heart conditions']},
{'Name': 'Finding strength and support through shared experiences', 'Description': 'Parents find comfort and healing in sharing their experiences with others who understand their situation. The act of listening to and discussing their stories provides emotional support and a sense of community, which is particularly valuable for those feeling isolated in their journey. They emphasize the importance of a strong support system during health crises.', 'Codes': ['Therapeutic value of shared experiences among parents', 'Willingness to support other parents post-surgery', 'The importance of a strong support system', 'Supportive community and shared experiences among parents']},
{'Name': "Managing constant vigilance and anxiety about the child's health", 'Description': "Parents express the need to constantly monitor their child's health and activities, balancing freedom with safety. They feel a strong responsibility to ensure their child's well-being, often leading to anxiety and a sense of isolation when others do not understand their concerns. This vigilance is driven by the fear of potential health crises.", 'Codes': ["Constant vigilance and monitoring of child's condition", 'Walking on eggshells due to constant worry', 'Daily fear of heart attacks and reassurance from medical professionals']},
{'Name': 'Desiring clear communication and emotional support from healthcare providers', 'Description': "Parents emphasize the importance of receiving clear communication and emotional support from medical professionals during critical decision-making moments. They value transparency in discussing their child's condition and treatment options, which helps them feel more secure in their decisions. This desire for open dialogue is crucial for navigating the complexities of their child's health.", 'Codes': ['Desire for honest communication from medical professionals', 'Emotional support from medical professionals during critical decisions', 'Desire for emotional support and better communication from doctors']},
{'Name': 'Struggling with parental guilt and decision-making', 'Description': "Parents express a profound sense of guilt regarding the decisions they make for their child's care. They feel torn between allowing their child to engage in activities and the fear of potential harm, leading to a constant emotional struggle. This guilt is compounded by the complexities of their child's medical condition and the decisions that must be made in the face of uncertainty.", 'Codes': ["Parental guilt over decision-making for their child's care", 'Concerns about being proactive in treatment decisions']},
{'Name': 'Experiencing relief and anxiety after diagnosis', 'Description': "Parents express a complex mix of relief and anxiety upon receiving their child's diagnosis. While the diagnosis provides clarity and answers to their child's symptoms, it also brings the weight of understanding the severity of the condition. This emotional rollercoaster reflects the journey from uncertainty to a new reality, where parents grapple with the implications of the diagnosis for their child's future.", 'Codes': ['Relief and anxiety following a diagnosis of AAOCA', 'Relief from diagnosis and clarity about the condition', 'Initial shock and fear upon diagnosis']},
{'Name': 'Making urgent decisions about surgery', 'Description': "Parents discuss the urgency surrounding surgical interventions for their children, often feeling that surgery is a necessity rather than an option. This urgency leads to quick decision-making, reflecting their desire to ensure their child's health and future. The emotional burden of these decisions is compounded by the fear of potential outcomes and the need for timely medical action.", 'Codes': ['The urgency of surgical intervention and its implications', 'Decisive action following identification of medical issues', 'Guilt and fear surrounding the decision for surgery']},
{'Name': 'Struggling with emotional burdens of decision-making', 'Description': "Parents articulate the emotional toll of making decisions about their child's surgery, often feeling overwhelmed by the weight of responsibility. The fear of negative outcomes and feelings of guilt complicate their decision-making process, leading to significant anxiety. This struggle reflects the challenge of balancing their child's well-being with their own emotional needs.", 'Codes': ['Emotional burden of decision-making and guilt', 'The burden of decision-making regarding surgery', 'Struggle with the decision-making process regarding surgery']},
{'Name': 'Seeking clarity and support in medical decisions', 'Description': "Parents emphasize the importance of clear communication and support from medical professionals during the decision-making process. They express a desire for comprehensive information and reassurance, often seeking second opinions to feel confident in their choices. This need for clarity reflects their commitment to making informed decisions for their child's health.", 'Codes': ['Navigating the complexities of medical decisions', 'Desire for clearer guidance during decision-making about surgery', 'Importance of clear communication from medical professionals']},
{'Name': "Coping with ongoing anxiety about child's health", 'Description': "Despite surgery, parents continue to experience anxiety regarding their child's health. They grapple with fears about potential complications and the implications of living with a heart condition. This ongoing vigilance reflects the emotional toll of caring for a child with serious health issues, as parents remain alert to any signs of distress.", 'Codes': ['Anxiety persists even after surgery', "Ongoing anxiety about child's heart condition and symptoms", "Living with uncertainty and fear regarding child's health"]},
{'Name': 'Desiring a normal life for their child post-surgery', 'Description': 'Parents express a strong desire for their child to lead a normal life after surgery, balancing their fears with the hope for happiness and normalcy. They reflect on the importance of allowing their child to participate in activities while managing the risks associated with their condition. This desire underscores the emotional complexity of their journey.', 'Codes': ['Desire for child to live a normal life post-surgery', 'Concerns about quality of life post-surgery', 'Navigating the balance between normalcy and caution']},
{'Name': 'Experiencing frustration with medical communication', 'Description': "Parents articulate their frustration with medical professionals who do not take their concerns seriously or provide clear information. This lack of acknowledgment leads to increased anxiety and confusion about their child's health. The struggle for effective communication highlights the emotional burden parents face in advocating for their child's needs.", 'Codes': ['Frustration with medical professionals and communication', 'Frustration with doctors not taking symptoms seriously', 'Desire for better communication and information from doctors']},
{'Name': 'Finding community and support among other parents', 'Description': 'Parents express a strong desire for community and connection with others who share similar experiences. They find comfort in discussing their journeys and seek support from online platforms and local groups. This sense of community helps alleviate feelings of isolation and provides reassurance during challenging times.', 'Codes': ['Finding support and community among other parents', 'Support from online communities and shared experiences', 'Desire for community and shared experiences among parents']},
{'Name': "Navigating the emotional toll of living with a child's condition", 'Description': "Parents describe the emotional ups and downs associated with their child's health condition, likening it to a rollercoaster. They experience moments of hope and reassurance, but also face fears and uncertainties about the future. This emotional landscape reflects the ongoing challenges of managing their child's health and well-being.", 'Codes': ['Emotional rollercoaster of hope and despair', "Emotional rollercoaster of living with a child's health condition", 'The emotional impact of annual medical evaluations']},
{'Name': 'Appreciating comprehensive care and support from medical teams', 'Description': "Parents express gratitude for the comprehensive care and support provided by interdisciplinary medical teams. They feel reassured by the collaborative approach to their child's treatment, which addresses the full spectrum of their diagnosis. This support fosters confidence in managing their child's health and navigating the complexities of care.", 'Codes': ['Support from interdisciplinary medical teams', 'Gratitude for comprehensive care and support', 'Importance of a coordinated support system in healthcare']},
{'Name': 'Advocating for informed care and support', 'Description': "Parents feel a strong sense of responsibility to advocate for their child's health, actively seeking information and resources. They emphasize the importance of knowledgeable medical professionals and the need for clear communication about their child's condition. This advocacy is driven by a desire to ensure the best possible care and outcomes, as well as to navigate the complexities of their child's health journey.", 'Codes': ['Empowerment through advocacy and information-seeking', 'Value of knowledgeable medical professionals in navigating care', 'Caution against misinformation from well-meaning sources', 'Desire for clear communication upon diagnosis', 'Trusting medical decisions and care plans', 'Seeking specialized care for complex conditions', 'Importance of specialized pediatric care in complex medical conditions']},
{'Name': 'Managing emotional turmoil and anxiety', 'Description': "Parents navigate a complex emotional landscape filled with anxiety, fear, and uncertainty regarding their child's health. They express feelings of shock and disbelief upon diagnosis, as well as ongoing concerns about long-term health. The emotional impact of surgery and the need for self-care after difficult conversations highlight the psychological toll of caring for a child with AAOCA.", 'Codes': ['Shock and disbelief upon diagnosis of AAOCA', 'Emotional impact of surgery on parents and children', 'Concerns about long-term health and asymptomatic conditions', 'Emotional turmoil surrounding emergency surgery experiences', 'Navigating the emotional rollercoaster of diagnosis and treatment', 'Psychological impacts post-surgery']},
{'Name': 'Striving for normalcy amidst health challenges', 'Description': "Parents express a strong desire for their children to return to a sense of normalcy after surgery. They strive to maintain typical family life and encourage their children to engage in activities they enjoy, despite the underlying health concerns. This desire reflects their commitment to supporting their children's interests while managing health limitations.", 'Codes': ['Desire for normalcy in daily life post-surgery', "Children's activities as a central focus of parental life", 'Limiting family activities to accommodate health concerns', 'Desire for autonomy balanced with safety for children']},
{'Name': 'Seeking community and peer support', 'Description': 'Parents highlight the importance of community support and peer connections for both themselves and their children. They express a strong desire for their children to connect with peers who have similar health experiences, believing that shared understanding fosters resilience. Additionally, parents appreciate the unexpected support from community members during crises.', 'Codes': ['Importance of peer support for children with similar conditions', 'Unexpected support from community members in crisis', 'Acknowledgment of shared experiences among parents', 'Gratitude for shared experiences and support from others']},
{'Name': 'Navigating healthcare complexities and decisions', 'Description': "Parents face significant challenges in navigating the complexities of healthcare decisions for their children. They express frustration with mixed messages from medical professionals, the urgency of treatment decisions, and the emotional burden of trusting varying opinions. This theme reflects the ongoing struggle to balance safety with normalcy in their child's life.", 'Codes': ['Diverse opinions on surgical necessity among medical professionals', 'Trusting the expertise of specialized providers for complex cases', 'Navigating uncertainty in treatment decisions for complex cases', 'Frustration with delayed diagnosis and medical responses']},
{'Name': 'Balancing health concerns with teenage independence', 'Description': 'Parents grapple with the challenge of allowing their teenagers to experience independence while managing health risks associated with their condition. They express concerns about social pressures, risky behaviors, and the need for open communication about health choices. This balancing act reflects their desire for their children to lead fulfilling lives while ensuring their safety.', 'Codes': ['Navigating teenage experimentation with a heart condition', 'Teenagers downplaying their heart condition', 'Concerns about social pressures and peer influence on health choices']},
{'Name': 'Desiring clear communication and understanding', 'Description': "Parents express a strong need for clear and consistent communication from healthcare providers regarding their child's condition and treatment options. They seek better explanations to alleviate uncertainty and anxiety about their child's health status, emphasizing the importance of effective communication in fostering trust and understanding.", 'Codes': ['Effective communication and involvement in care decisions', 'Desire for clearer understanding of medical conditions and decisions', 'Desire for clear communication from medical professionals about limitations']},
{'Name': 'Emphasizing the importance of awareness and education', 'Description': 'Parents advocate for increased awareness and education regarding heart conditions in children. They emphasize the need for better screening and understanding of AAOCA to prevent late diagnoses and ensure children are cleared for sports and activities safely. This desire reflects their commitment to improving outcomes for their children and others facing similar challenges.', 'Codes': ['The importance of awareness and education about heart conditions', 'Struggles with misdiagnosis and delayed treatment']},
{'Name': 'Managing the emotional toll of activity restrictions', 'Description': 'Parents struggle with the emotional burden of enforcing activity restrictions on their children post-diagnosis. They express feelings of trauma and helplessness as they navigate the challenges of explaining these limitations to their children and family. The drastic shift from an active lifestyle to enforced inactivity creates significant emotional strain, leading to feelings of frustration and sadness.', 'Codes': ["Struggles of managing a child's activity restrictions", "Emotional impact of a child's health condition on family dynamics", 'Balancing fear and normalcy in parenting']},
{'Name': 'Advocating for timely and effective medical care', 'Description': "Parents demonstrate a strong determination to advocate for their child's health, often pushing for further tests and consultations when initial assessments fall short. They express frustration with inconsistent medical opinions and the urgency they feel for timely interventions. This advocacy reflects their commitment to ensuring their child receives the best possible care amidst a complex healthcare landscape.", 'Codes': ['Determination to advocate for proper medical care', 'Frustration with inconsistent medical opinions and lack of urgency', 'Desperation for timely intervention and resolution', 'Navigating the healthcare system for second opinions', 'The necessity of seeking multiple medical opinions']},
{'Name': 'Finding relief in competent medical support', 'Description': "Parents express relief and confidence upon finding a skilled medical team to handle their child's surgery. The acknowledgment of the doctor's expertise and the quick scheduling of surgery provide a sense of security during a tumultuous time. This theme highlights the importance of trust in medical professionals and the emotional comfort that comes from competent care.", 'Codes': ['Relief in finding a competent medical team', 'Trust in medical professionals despite differing opinions', 'Importance of finding experienced and specialized surgeons']},
{'Name': 'Coping with the emotional aftermath of surgery', 'Description': "Parents highlight the profound emotional challenges their children face during recovery from surgery. They express concerns about their children's mental health, including feelings of depression and anxiety, and emphasize the need for mental health support. This theme underscores the importance of addressing emotional well-being as part of the recovery process.", 'Codes': ['The emotional impact of surgery on children and families', 'The need for mental health support post-surgery', 'Emotional processing and recovery post-surgery', "Impact of health issues on children's mental health"]},
{'Name': 'Navigating fears and anxieties about health', 'Description': "Parents express deep concerns about their children's heightened fears and anxieties regarding their health, particularly in relation to symptoms like chest pain. They recognize the need for mental health support to help their children cope with these feelings. This theme reflects the emotional burden parents carry as they manage their children's health-related fears.", 'Codes': ["Children's fear and anxiety about health issues", 'Managing anxiety and panic attacks related to heart condition', "Struggles with managing children's health-related fears"]},
{'Name': 'Seeking community and support from others', 'Description': 'Parents emphasize the importance of connecting with other families facing similar challenges. This connection provides emotional support and validation, helping them navigate their own experiences. The shared understanding fosters a sense of community, alleviating feelings of isolation and loneliness during difficult times.', 'Codes': ['Importance of social support and community', 'The importance of connecting with other parents in similar situations', 'Lack of local support and community resources']},
{'Name': 'Struggling with the complexities of medical communication', 'Description': "Parents describe the challenges they face in communicating their children's medical needs to healthcare providers and schools. They often feel misunderstood or dismissed, which adds to their stress as they advocate for their children's needs. This theme highlights the emotional toll of navigating complex medical communication.", 'Codes': ['Navigating medical bureaucracy and communication challenges', 'Frustration with healthcare processes and accessibility', 'The challenge of communicating complex medical information to children']},
{'Name': 'Balancing hope and fear in decision-making', 'Description': "Parents navigate the emotional rollercoaster of diagnosis and treatment decisions, balancing hope for recovery with fear of complications. They express anxiety over the rapid pace of medical decisions and the uncertainty surrounding their child's health. This theme captures the emotional complexity of making critical decisions in a high-stakes environment.", 'Codes': ['Emotional rollercoaster of diagnosis and treatment decisions', 'Factors influencing the decision-making process for surgery', 'Struggles in family decision-making regarding surgery']},
{'Name': 'Empowering through education and advocacy', 'Description': "Parents find empowerment through education about their child's condition and advocacy within the community. They highlight the importance of being open about their experiences and connecting with others facing similar challenges. This empowerment fosters a sense of agency and resilience in both parents and children.", 'Codes': ['Empowerment through education and advocacy', "Finding empowerment through advocacy for their child's health", 'The importance of reliable medical information and resources']},
{'Name': 'Focusing on child’s health and well-being during crises', 'Description': "In the face of their child's health challenges, parents become intensely focused on ensuring their child's well-being. This hyper-focus often leads to a sense of clarity and determination, allowing them to prioritize their child's needs above all else. Parents express a strong commitment to making decisions that align with their child's desires, often disregarding external opinions to ensure their child feels supported and cared for.", 'Codes': ['Hyper-focus on child’s health and well-being during crisis', 'Encouraging child involvement in medical decisions', 'The emotional impact of the diagnosis and treatment journey']},
{'Name': 'Seeking clarity and support in navigating medical complexities', 'Description': "Parents express a strong desire for clear communication and comprehensive information from healthcare providers regarding their child's condition and treatment options. They often feel overwhelmed by the complexities of insurance processes and conflicting medical opinions, leading to anxiety and confusion. Gratitude for support from partners and medical professionals is also emphasized, as it helps them navigate these challenges.", 'Codes': ['Navigating insurance complexities during medical treatment', "Navigating complex insurance processes during a child's medical care", 'Desire for clear communication from medical professionals', 'Navigating conflicting medical opinions about surgery for AAOCA', 'Desire for more information and clarity about the condition']},
{'Name': 'Coping with emotional turmoil and mental health challenges', 'Description': "Parents highlight the emotional toll that their child's health condition takes on the entire family. They express concerns about their child's mental health, particularly in relation to anxiety, depression, and feelings of isolation. The need for counseling and support for both the child and the family is emphasized, as parents recognize the importance of addressing emotional well-being alongside physical health.", 'Codes': ['The importance of counseling for families dealing with AAOCA', "Impact of AAOCA on children's mental health and social life", "The emotional impact of a child's health condition on parents", 'The impact of mental health struggles on children with AAOCA', 'Finding joy in alternative activities during restrictions']},
{'Name': 'Balancing caregiving responsibilities with personal well-being', 'Description': 'Parents share their experiences of managing work commitments while caring for their recovering child. They often struggle to find a balance between professional obligations and the demands of caregiving, leading to feelings of guilt and stress. Some parents attempt to use work as a distraction from emotional turmoil, only to realize that this approach can be counterproductive to their well-being.', 'Codes': ['Using work as a distraction from emotional turmoil', 'Balancing work and caregiving responsibilities during recovery']},
{'Name': 'Addressing the long-term implications of AAOCA', 'Description': "Parents express ongoing concerns about the long-term health monitoring required for their children with AAOCA, especially as they transition to adult care. They seek reassurance about their child's future health outcomes and express anxiety about potential complications. The desire for more research and awareness regarding right coronary artery anomalies is also highlighted, reflecting their need for better understanding and support.", 'Codes': ['Long-term health monitoring and transition to adult care', 'Concerns about long-term outcomes and future health', 'Need for more research on right coronary artery anomalies']},
{'Name': 'Finding community and shared experiences among parents', 'Description': "Parents find comfort and support in connecting with others who share similar experiences. They appreciate the opportunity to discuss their challenges and learn from each other's journeys, which helps alleviate feelings of isolation. The relief of shared experiences and the importance of community support are emphasized as vital components in navigating the emotional landscape of raising a child with AAOCA.", 'Codes': ['Finding support and community among other parents', 'The relief of shared experiences among parents', 'The value of community support for parents']},
{'Name': 'Managing the emotional impact of surgery decisions', 'Description': "The emotional toll of making surgery decisions for their children is evident among parents. They express feelings of fear, anxiety, and relief as they navigate the complexities of their child's heart condition and the associated surgical interventions. Parents emphasize the importance of trust in medical professionals and the need for thorough evaluations to help them feel confident in their decisions.", 'Codes': ['Emotional impact of surgery decisions on families', 'Trust in medical professionals and their thoroughness during care', 'The emotional impact of surgery on children and families']},
{'Name': "Understanding and addressing children's health limitations", 'Description': "Parents face challenges in helping their children understand their health limitations and the importance of following medical advice. They struggle to communicate the seriousness of their child's condition while also addressing the emotional impact of restrictions on activities. The need for clear and compassionate communication from healthcare providers is emphasized to facilitate better understanding and compliance.", 'Codes': ['Challenges in helping child understand health limitations', "Child's fear and anxiety about health condition", 'The challenge of explaining the diagnosis to young children']},
{'Name': 'Expressing gratitude for compassionate medical professionals', 'Description': "Parents express deep appreciation for medical professionals who demonstrate compassion and understanding during their child's health journey. They value doctors who take the time to explain diagnoses thoroughly and provide emotional support, which alleviates some of the stress associated with their child's condition. This gratitude fosters a sense of trust and reassurance in the healthcare process.", 'Codes': ['Appreciation for compassionate medical professionals', 'Desire for clear communication and information from doctors', 'Desire for a supportive and understanding healthcare provider', 'Positive experiences with medical staff and care', 'The role of empathy and understanding in pediatric care', 'The importance of a trusting relationship with the doctor']},
{'Name': "Managing ongoing stress and anxiety about children's health", 'Description': "Parents report persistent stress and anxiety regarding their children's health, particularly in the face of potential complications. They express concerns about managing daily life with their child's condition, the emotional toll of waiting for surgery, and the fear of health crises. This ongoing anxiety shapes their daily routines and interactions with their children.", 'Codes': ["Concerns about managing daily life with a child's condition", 'Fear and anxiety about potential health issues for children', "Ongoing stress and anxiety about children's health", 'Emotional toll of waiting for surgery and uncertainty', 'Emotional challenges during the recovery process', 'Emotional toll of living with a rare heart condition']},
{'Name': 'Seeking clear communication and comprehensive information', 'Description': "Parents emphasize the necessity of clear, concise communication from healthcare providers regarding their child's condition and treatment options. They express a strong desire for comprehensive information about AAOCA, including potential risks and long-term outcomes. This need for clarity is crucial for making informed decisions and alleviating anxiety.", 'Codes': ['Desire for clear communication and information from doctors', 'Desire for comprehensive information about AAOCA and its implications', 'Frustration with unclear medical information and guidance', 'Desire for better communication and follow-up from doctors', 'Need for structured guidance in discussing health issues with children']},
{'Name': 'Balancing hope and fear in parenting a child with health challenges', 'Description': "Parents navigate a complex emotional landscape, balancing hope for their child's future with fears about their health and well-being. They acknowledge the silver linings while also recognizing the challenges that come with their child's condition. This balancing act influences their approach to parenting and decision-making.", 'Codes': ['Balancing hope and fear in parenting a child with AAOCA', 'Living in the present amidst health concerns', 'Fostering resilience in children with health challenges', 'Impact of diagnosis on self-esteem and future aspirations']},
{'Name': 'Establishing a support network for emotional well-being', 'Description': "Parents highlight the importance of emotional support and the value of connecting with other families facing similar challenges. They express a desire for shared experiences and understanding, which can help alleviate feelings of isolation and provide comfort during difficult times. This support network is vital for coping with the emotional toll of their child's condition.", 'Codes': ['Need for emotional support and shared experiences among parents', 'Need for peer support and shared experiences', 'The importance of community and shared experiences', 'Need for peer support among parents']},
{'Name': "Trusting instincts and advocating for children's health", 'Description': "Parents emphasize the importance of trusting their instincts when it comes to medical decisions for their children. They advocate for further testing and seek out specialists when they feel something is not right. This self-advocacy is crucial in navigating the complexities of their child's healthcare journey.", 'Codes': ['Importance of maternal instinct in seeking medical help', 'Trusting instincts in medical decisions', 'Finding the right specialist for care', 'The necessity of self-advocacy and informed decision-making', 'The importance of seeking second opinions for medical decisions']},
{'Name': 'Navigating the complexities of treatment decisions', 'Description': 'Parents face challenging decisions about treatment options for their child, often feeling torn between the risks of surgery and the uncertainty of leaving the condition untreated. They express frustration with conflicting medical opinions and the emotional burden of making informed decisions amidst uncertainty.', 'Codes': ['Difficult decision-making regarding treatment options', 'Navigating conflicting medical opinions and recommendations', 'The uncertainty of diagnosis and treatment options for AAOCA', 'Frustration with delays in follow-up care and specialist access']},
{'Name': 'Concerns about long-term health and lifestyle choices', 'Description': "Parents express worries about their child's long-term health, particularly regarding lifestyle choices that could impact their condition. They highlight the importance of monitoring factors like cholesterol and the need for guidance on healthy living to prevent future complications. This concern is compounded by the uncertainty of their child's future health outcomes.", 'Codes': ['Concerns about long-term health and lifestyle choices post-surgery', "Children's perception of their health risks and limitations", 'Concerns about physical activity and safety', 'Navigating the balance between normalcy and caution in sports activities']}]

In [ ]:
len(groups)

##### previous  theme

In [ ]:
final_themes={'major_themes': [{'name': "Seeking clarity and reassurance about my child's health journey",
   'description': "Parents express a strong desire for clarity and reassurance regarding their child's health and future. They often feel overwhelmed by uncertainties and seek comprehensive information from medical professionals about long-term outcomes, treatment options, and follow-up care. This need for clarity is coupled with anxiety about potential complications and the emotional toll of navigating a complex medical landscape.",
   'codes': ["Desire for clarity about the child's future health",
    'Fear of long-term outcomes after surgery',
    'Concerns about follow-up care and future health',
    'Desire for statistical data on condition outcomes',
    'Desire for comprehensive data on surgery outcomes for reassurance',
    'Desire for clear communication and information from healthcare providers']},
  {'name': "Balancing normalcy and protection in my child's life",
   'description': 'Parents grapple with the desire for their children to lead active, normal lives while simultaneously feeling the need to protect them from potential health risks. This internal conflict manifests in their decisions about allowing participation in sports and social activities, reflecting a deep-seated wish for their children to experience typical childhood moments without the shadow of their health condition.',
   'codes': ["Desire for normalcy in the child's life despite health challenges",
    "Desire for normalcy in child's life post-surgery",
    "Fear of limitations on child's activities post-diagnosis",
    "Struggles with restrictions on child's activities"]},
  {'name': "Coping with emotional turmoil and uncertainty about my child's future",
   'description': "Parents articulate the emotional burden and turmoil they experience while caring for a child with AAOCA. They navigate feelings of anxiety, guilt, and helplessness, particularly during critical moments such as receiving the diagnosis or making surgical decisions. This emotional landscape is compounded by the unpredictability of their child's health, leading to a constant state of vigilance and worry about potential health crises.",
   'codes': ['Emotional turmoil over surgical decisions',
    'Emotional turmoil from vague medical explanations',
    "Anxiety about the child's health and future",
    'Daily fear of heart attacks affecting family life',
    'Emotional burden of parenting a child with AAOCA',
    "Overwhelming anxiety about child's health and future"]},
  {'name': 'Desiring emotional support and understanding from healthcare providers',
   'description': "Parents emphasize the importance of receiving emotional support and clear communication from healthcare professionals during their child's health journey. They express frustration when medical explanations lack sensitivity or clarity, highlighting the need for compassionate care that addresses both the technical and emotional aspects of their child's condition.",
   'codes': ['Emotional support from medical professionals during crisis',
    'Desire for more emotional support from healthcare providers',
    'Need for connection with other parents in similar situations',
    'Desire for honest communication from medical professionals']},
  {'name': 'Experiencing relief and gratitude for successful interventions',
   'description': "Parents convey profound relief and gratitude when their child's medical interventions lead to positive outcomes. They recognize the importance of timely surgeries and effective treatments that allow their children to thrive. This sense of relief is often accompanied by a celebration of their child's resilience and achievements, reflecting a hopeful outlook amidst the challenges of managing a serious health condition.",
   'codes': ['Relief and gratitude for successful medical interventions',
    'Joy in family time amidst health challenges',
    "Pride in child's resilience and achievements",
    'Relief from reassurance by medical professionals']},
  {'name': 'Living with constant vigilance and fear of health crises',
   'description': "Parents live in a state of heightened alertness, constantly monitoring their child's activities and health status due to the fear of sudden health crises. This vigilance is driven by the knowledge of potential risks associated with their child's condition, leading to anxiety about everyday activities. The emotional toll of this constant worry shapes their daily lives and interactions.",
   'codes': ['Constant vigilance and fear of sudden health crises',
    "Constant vigilance and monitoring of child's activities",
    "Awareness of child's health status and emotional impact"]},
  {'name': "Struggling with guilt and self-blame regarding my child's condition",
   'description': "Parents often grapple with feelings of guilt and self-blame related to their child's health condition. They reflect on their role in their child's life and question their decisions, fearing they may have contributed to the child's health issues. This emotional struggle is intensified by comparisons with siblings and the desire to be proactive in their child's care.",
   'codes': ['Guilt over potential inaction and regret',
    "Guilt and self-blame for the child's condition",
    'Parental guilt over decision-making for their child']},
  {'name': 'Desiring positive messaging about health outcomes',
   'description': 'Parents express a strong desire for clear, positive messaging regarding the outcomes for children diagnosed with AAOCA. They want to counteract negative assumptions and emphasize that children can lead successful lives despite their challenges, advocating for a narrative of resilience.',
   'codes': ['Desire for positive messaging about AAOCA outcomes',
    'Need for reassurance during initial diagnosis']},
  {'name': 'Feeling overwhelmed by the demands of caregiving',
   'description': "Parents express feelings of being overwhelmed by the constant demands of caring for their children with AAOCA. Their lives revolve around their children's needs, leading to exhaustion and a loss of personal identity as they prioritize their children's health above their own.",
   'codes': ['Feeling overwhelmed by the demands of caring for children with AAOCA',
    "Emotional toll of living with a child's chronic condition"]},
  {'name': 'Building a supportive community with other parents',
   'description': "Parents actively seek support and community from others who have similar experiences. They turn to social media and support groups to share information, gain insights, and find comfort in knowing they are not alone in their struggles. This connection helps them navigate the complexities of their child's condition.",
   'codes': ['Seeking support and community from other parents',
    'The importance of connecting with other parents in similar situations',
    'Desire for community support and shared experiences']},
  {'name': 'Managing the emotional impact of surgery on my child',
   'description': "Parents reflect on the profound emotional toll that surgery has on their children. They describe feelings of loss, depression, and a sense of disconnection from peers due to the abrupt changes in their child's life. The transition from a typical childhood to one marked by medical challenges creates a significant emotional burden.",
   'codes': ['Emotional impact of surgery on the child',
    'Concerns about social reintegration and peer relationships',
    'Fear of long-term health consequences']},
  {'name': 'Advocating for awareness and understanding of heart conditions',
   'description': 'Parents feel a strong responsibility to raise awareness about heart conditions and the importance of early detection. They engage in advocacy efforts to educate others about the risks and necessary precautions for children with heart anomalies, reflecting their commitment to improving outcomes for others.',
   'codes': ['Advocacy for awareness of heart conditions',
    'Navigating medical uncertainty and trust']}]}

In [ ]:
[i['name'] for i in final_themes['major_themes']]

##### coverage

In [ ]:
import os
import json
import numpy as np
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [ ]:
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI

In [ ]:
open_api_key = ...
os.environ["OPENAI_API_KEY"]=open_api_key
model_name = "gpt-4o"
model = ChatOpenAI(model_name = model_name, temperature = 0)

In [ ]:
evaluation_prompt1 = ChatPromptTemplate.from_template("""


1. Read the orginal data {groups}
2. Evaluate the generated theme names based on the coverage criteria

## Coverage:
- The themes comprehensively captures the key aspects of parents’ lived experiences
- The themes are present broadly within the transcripts across multiple participants and evidence of the theme exists across multiple excerpts of data.  
- The themes should not represent that experience of only one or two participants even if the theme is discussed frequently by those one or two participants.

THEMES: {themes}
groups: {groups}

Return an evaluation in valid JSON format with the following details:
- **Score (1-5)**: A score representing the quality (5 means very good, 1 means not good) 
- **Feedback**: A detailed explanation on why this score was given and give specific suggestions on how to add important themes from orginal data {groups} from parents’ lived experiences


Return an evaluation feedback in valid JSON format 
Ensure the response is in valid JSON format, with no markdown or extra text.
""")

evaluation_chain1 = LLMChain(llm=model, prompt=evaluation_prompt1)

def evaluate_themes1(themes, groups):
    response = evaluation_chain1.run({
        "themes": json.dumps(themes),
        "groups": json.dumps(groups)
    })
    cleaned_response = response.replace("```json\n", "").replace("\n```", "")
    return json.loads(cleaned_response)  # Convert GPT response to a Python dictionary

In [ ]:
# 🔹 **Step 3: Regenerate Themes Using Feedback**
feedback_prompt1 = ChatPromptTemplate.from_template("""

1. {groups} is the orginal data to summarize {themes}
2. Review the Evaluation Feedback {evaluation} about {themes}
3. Add missing themes mentioned in the {evaluation} to the {themes}

Change based on the evulation
The final themes should be concise and written in sentence case, with each theme containing 8 to 15 words. Each theme should also include a description of 60 to 80 words that captures the essence of the theme, along with a list of all relevant code names.
Do not use 'navigate' in themes, please no 'navigate'

- **Evaluation Feedback**: {evaluation}
- **THEMES**: {themes}
- **groups**: {groups}

Return a **valid JSON list** of improved themes without any markdown or extra text.
""")

feedback_chain = LLMChain(llm=model, prompt=feedback_prompt1)

def regenerate_themes1(evaluation,themes,groups):
    response = feedback_chain.run({
    "evaluation": json.dumps(evaluation),
    "themes": json.dumps(themes),
    "groups": json.dumps(groups)
    })
#     print(response)
    cleaned_response = response.replace("```json\n", "").replace("\n```", "")
    return json.loads(cleaned_response)  # Convert GPT response to a Python dictionary

##### actionability

In [ ]:
evaluation_prompt3 = ChatPromptTemplate.from_template("""

Evaluate the generated theme names: {themes} based on the following criteria

## Actionability:
- Have a single concept in each one theme
- The themes in {themes} can provide clear, specific, and meaningful insights that can inform interventions, resources, or research.
- It avoids being too broad, vague, or overly general. 
- The themes in {themes} should not only capture general ideas such as “normalcy” or “emotional impact” 


THEMES: {themes}

Return an overall evaluation in valid JSON format with the following details:
- **Score (1-5)**: A score representing the quality (5 means very good, 1 means not good)
- **Feedback**: A detailed explanation on why this score was given and give specific suggestions how to improve the theme names



Return an overall evaluation feedback in valid JSON format 
Ensure the response is in valid JSON format, with no markdown or extra text.
""")

evaluation_chain3 = LLMChain(llm=model, prompt=evaluation_prompt3)

def evaluate_themes3(themes):
    response = evaluation_chain3.run({
        "themes": json.dumps(themes)
    })
    cleaned_response = response.replace("```json\n", "").replace("\n```", "")
    return json.loads(cleaned_response)  # Convert GPT response to a Python dictionary


In [ ]:
# 🔹 **Step 3: Regenerate Themes Using Feedback**
feedback_prompt3 = ChatPromptTemplate.from_template("""

Review the Evaluation Feedback {evaluation}, and split into different themes as mentioned in the evaluation

Write the new imrpoved Themes: The final themes should be concise and written in sentence case, with each theme containing 8 to 15 words. Each theme should also include a description of 60 to 80 words that captures the essence of the theme, along with a list of all relevant code names.

- **Evaluation Feedback**: {evaluation}
- **THEMES**: {themes}

Return a **valid JSON list** of improved themes without any markdown or extra text.
""")

feedback_chain = LLMChain(llm=model, prompt=feedback_prompt3)

def regenerate_themes3(evaluation,themes):
    response = feedback_chain.run({
    "evaluation": json.dumps(evaluation),
    "themes": json.dumps(themes)
    })
#     print(response)
    cleaned_response = response.replace("```json\n", "").replace("\n```", "")
    return json.loads(cleaned_response)  # Convert GPT response to a Python dictionary

##### distinctiveness

In [ ]:
evaluation_prompt2 = ChatPromptTemplate.from_template("""

Evaluate the generated theme names: {themes} based on the following criteria

## Distinctiveness:
- The themes in {themes} are clearly distinct with each other in theme names
- There should be no overlaps or redundancies that should be merged or refined to improve uniqueness of a theme.

THEMES: {themes}

Return an evaluation in valid JSON format with the following details:
- **Score (1-5)**: A score representing the quality (5 means very good, 1 means not good)
- **Feedback**: A detailed explanation on why this score was given and list specific suggestions how to improve the theme names



Return an overall evaluation feedback in valid JSON format 
Ensure the response is in valid JSON format, with no markdown or extra text.
""")

evaluation_chain2 = LLMChain(llm=model, prompt=evaluation_prompt2)

def evaluate_themes2(themes):
    response = evaluation_chain2.run({
        "themes": json.dumps(themes)
    })
    cleaned_response = response.replace("```json\n", "").replace("\n```", "")
    return json.loads(cleaned_response)  # Convert GPT response to a Python dictionary


In [ ]:
# 🔹 **Step 3: Regenerate Themes Using Feedback**
feedback_prompt2 = ChatPromptTemplate.from_template("""

Review the Evaluation Feedback {evaluation}, and combine the repeated themes in {themes}

Write the new imrpoved Themes: The final themes should be concise and written in sentence case, with each theme containing 8 to 15 words. Each theme should also include a description of 60 to 80 words that captures the essence of the theme, along with a list of all relevant code names.
Do not use 'navigate' in themes

- **Evaluation Feedback**: {evaluation}
- **THEMES**: {themes}

Return a **valid JSON list** of improved themes without any markdown or extra text.
""")

feedback_chain = LLMChain(llm=model, prompt=feedback_prompt2)

def regenerate_themes2(evaluation,themes):
    response = feedback_chain.run({
    "evaluation": json.dumps(evaluation),
    "themes": json.dumps(themes)
    })
#     print(response)
    cleaned_response = response.replace("```json\n", "").replace("\n```", "")
    return json.loads(cleaned_response)  # Convert GPT response to a Python dictionary

##### relevance

In [ ]:
evaluation_prompt4 = ChatPromptTemplate.from_template("""

Evaluate the generated theme names: {themes} based on the following criteria

## Relevance:
- The themes in {themes} accurately reflects parents’ lived experiences, concerns, and needs. They need to distinguish with child/patient feelings, concerns and experience.
Here are a few examples of parent vs. patient/child themes/outcomes. 

Parent Outcomes
Parents report they feel limited by their child’s diagnosis.
Parents report they have PTSD from their child’s experience.
Parents report being distressed by the uncertainty of treatment choices.
Parents report needing more social connections.
Patient/Child Outcomes
Parents report they feel their child is limited by the child’s diagnosis.
Parents report their child has PTSD from their child’s experience.
Parents report their child is distressed by the uncertainty of treatment choices.
Parents report their child needs more social connections.


- It avoid unnecessary emotional framing and interpretation if not supported by the data.
- The themes in {themes} is related to the experience of having a child with a diagnosis of AAOCA.


THEMES: {themes}

Return an overall evaluation in valid JSON format with the following details:
- **Score (1-5)**: A score representing the quality (5 means very good, 1 means not good)
- **Feedback**: A detailed explanation on why this score was given and give suggestions on deleting irrelevant information



Return an overall evaluation feedback in valid JSON format 
Ensure the response is in valid JSON format, with no markdown or extra text.
""")

evaluation_chain4 = LLMChain(llm=model, prompt=evaluation_prompt4)

def evaluate_themes4(themes):
    response = evaluation_chain4.run({
        "themes": json.dumps(themes)
    })
    cleaned_response = response.replace("```json\n", "").replace("\n```", "")
    return json.loads(cleaned_response)  # Convert GPT response to a Python dictionary


In [ ]:
# 🔹 **Step 3: Regenerate Themes Using Feedback**
feedback_prompt4 = ChatPromptTemplate.from_template("""

1. Review the Evaluation Feedback {evaluation}, 
2. delete irrelvant themes that mentioned in the {evaluation}
3. change to parents' perspectives

Write the new imrpoved Themes: The final themes should be concise and written in sentence case, with each theme containing 8 to 15 words. Each theme should also include a description of 60 to 80 words that captures the essence of the theme, along with a list of all relevant code names.
Do not use 'navigate' in themes
Do not use 'inlcuding' in themes

- **Evaluation Feedback**: {evaluation}
- **THEMES**: {themes}

Return a **valid JSON list** of improved themes without any markdown or extra text.
""")

feedback_chain = LLMChain(llm=model, prompt=feedback_prompt4)

def regenerate_themes4(evaluation,themes):
    response = feedback_chain.run({
    "evaluation": json.dumps(evaluation),
    "themes": json.dumps(themes)
    })
#     print(response)
    cleaned_response = response.replace("```json\n", "").replace("\n```", "")
    return json.loads(cleaned_response)  # Convert GPT response to a Python dictionary

In [ ]:
evaluation1 = evaluate_themes1(final_themes, groups)
print(evaluation1)

In [ ]:
re_themes1 = regenerate_themes1(evaluation1,final_themes,groups)
print(len(re_themes1))
print([i['name'] for i in re_themes1])

In [ ]:
evaluation3 = evaluate_themes3(re_themes1)
print(evaluation3)

In [ ]:
re_themes2 = regenerate_themes3(evaluation3,re_themes1)

In [ ]:
print(len(re_themes2))
print([i['name'] for i in re_themes2])

In [ ]:
evaluation2 = evaluate_themes2(re_themes2)
print(evaluation2)

In [ ]:
re_themes3 = regenerate_themes2(evaluation2,re_themes2)
print(len(re_themes3))
print([i['name'] for i in re_themes3])

In [ ]:
evaluation4 = evaluate_themes4(re_themes3)
print(evaluation4)

In [ ]:
re_themes4 = regenerate_themes4(evaluation4,re_themes3)
print(len(re_themes4))
print([i['name'] for i in re_themes4])

In [ ]:
re_themes4

In [ ]:
[i['name'] for i in re_themes4]

In [ ]:
evaluation1 = evaluate_themes1(re_themes4, groups)
print(evaluation1)

In [ ]:
re_themes1 = regenerate_themes1(evaluation1,re_themes4,groups)
print(len(re_themes1))
print([i['name'] for i in re_themes1])

In [ ]:
evaluation3 = evaluate_themes3(re_themes1)
print(evaluation3)

In [ ]:
re_themes2 = regenerate_themes3(evaluation3,re_themes1)

In [ ]:
print(len(re_themes2))
print([i['name'] for i in re_themes2])

In [ ]:
evaluation2 = evaluate_themes2(re_themes2)
print(evaluation2)

In [ ]:
evaluation2

In [ ]:
re_themes3 = regenerate_themes2(evaluation2,re_themes2)
print(len(re_themes3))
print([i['name'] for i in re_themes3])

In [ ]:
evaluation4 = evaluate_themes4(re_themes3)
print(evaluation4)

In [ ]:
re_themes4 = regenerate_themes4(evaluation4,re_themes3)
print(len(re_themes4))
print([i['name'] for i in re_themes4])

In [ ]:
re_themes4

In [ ]:
[i['name'] for i in re_themes4]

In [ ]:
['Seeking clarity and reassurance through multiple medical opinions',
 'Desiring comprehensive data and advancements in medical technology',
 "Balancing normalcy and protection in child's life",
 'Living with constant vigilance and fear of health crises',
 'Desiring emotional support from healthcare providers',
 'Experiencing relief and gratitude for successful interventions',
 "Struggling with guilt and self-blame regarding child's condition",
 'Desiring positive messaging about health outcomes',
 'Feeling overwhelmed by the demands of caregiving',
 'Building a supportive community with other parents',
 'Advocating for awareness and understanding of heart conditions',
 'Addressing frustration with the healthcare system',
 "Including long-term concerns about child's health"]

##### evaluation

##### similarity of description first sentence

In [ ]:
human_themes=[{'Theme': 'Clarity of potential risks and outcomes',
  'Description': 'While acknowledging the current challenges of robustly determining risk and outcome potentials in this diagnosis, parents described the uncertainty surrounding these as a primary driver of stress and anxiety related to the condition.',
  'Codes': ['Fear of cardiac events',
   'Uncertainty of future/what’s to come',
   'Parents always on',
   'Uncertainty of symptoms',
   'Impact of clinician placed activity restrictions',
   'Parent placed activity restrictions',
   'Searching for your own answers',
   'Trying to understand medical literature',
   'Lack of long-term outcomes data']},
 {'Theme': 'Freedom from hypervigilance related to the condition',
  'Description': 'A dominating sense of hypervigilance was described by parents with constant worry about cardiac events, always feeling they needed to monitor their child, and in some cases imposing activity restrictions on their child. This hypervigilance can at times be all consuming and is complicated by uncertainty around potential symptoms and their meaning.',
  'Codes': ['Fear of cardiac events',
   'Parents always on',
   'Uncertainty of symptoms',
   'Parent placed activity restrictions']},
 {'Theme': 'The diagnosis given in a compassionate and empathic way',
  'Description': 'While the diagnosis experience was minimally discussed by the individuals with AAOCA, the experience was quite impactful to parents who frequently described it as traumatic, frightening, and overwhelming. However, many parents spoke about the benefits of feeling they had a relationship with the clinician delivering the diagnosis and the positive impact a empathic delivery of the diagnosis had on the experience.',
  'Codes': ['Diagnosis as traumatic and frightening',
   'Diagnosis/information overwhelming',
   'Importance of clinician relationships and interactions',
   'Developmentally appropriate ways to tell children']},
 {'Theme': 'A sense of control over the future',
  'Description': 'Receiving the diagnosis of AAOCA for their child wrested any feeling of control the parent had over their child’s future. The desire for a feeling of some control was seen in parents shielding their children from the diagnosis when they are younger, restricting activities in the absence of clinician recommendation for restriction, and fear around their child’s transition to adulthood and greater self-management.\n',
  'Codes': ['Lack of control over outcomes for child',
   'Uncertainty of symptoms',
   'Parent placed activity restrictions',
   'Transition to adulthood',
   'Shielding kids from the diagnosis',
   'Planning for adverse events',
   'Lack of long-term outcomes data']},
 {'Theme': 'Being heard and taken seriously by clinicians',
  'Description': 'As with individuals with AAOCA, parents expressed the importance of being heard and taken seriously by clinicians. However, this was driven by different points in the care journey and predominantly revolved around the challenging diagnostic journey with frequent dismissals and misdiagnoses and the sense that as the parent, they were the ones who were responsible for driving testing and care forward.',
  'Codes': ['Frequent challenges within the diagnostic journey',
   'Parents driving care/work up/attention forward']},
 {'Theme': 'Individualized support for management decision-making',
  'Description': 'Similar to individuals with a diagnosis of AAOCA, parents desire individualized management for their child’s specific situation. The perception of management being decided in a “textbook” way, as described by parents lead many parents to immerse themselves in trying to understand complex medical literature  as they felt it was their responsibility to determine the “right” management strategy for their child.',
  'Codes': ['Decision between intervention and limitation',
   'Parents’ responsibility to make the “right” choice',
   'Finding what you want',
   'Different opinions form different clinicians',
   'Trying to understand medical literature',
   'Impact of clinician placed activity restrictions']},
 {'Theme': 'Receiving support from others',
  'Description': 'As in individuals with a diagnosis of AAOCA, parents expressed the isolation that can come following the diagnosis. However, in parents, this only manifested in feeling that others don’t truly understand their experience having a child with the diagnosis and did not include the loss of connection to others due to altered activities. Parents endorsed the benefits of a close group of friends and family who they could lean on for support as well as making connections with other families who have the same experience. ',
  'Codes': ['Importance of support from family and friends',
   'Connections with others in similar situations – Parents',
   'Connections with others in similar situations – Child',
   'Comforting influence of spirituality']},
 {'Theme': 'Being appropriately informed',
  'Description': 'Parents also endorsed the importance of feeling appropriately informed throughout the journey. While this desire was present broadly across the journey it particularly manifested around the time of diagnosis and understanding the implications of the diagnosis on them and their child.',
  'Codes': ['Searching for answers on own',
   'Trying to understand medical literature',
   'Diagnosis/information overwhelming',
   'Lack of long-term outcomes data']},
 {'Theme': 'Partnership with the care team',
  'Description': 'Being a partner with the care team was also important for parents. They did not want to just be seen as the decision maker for the child but rather a care team member to contribute their knowledge and expertise of their child and the family in the management and care discussions and decisions. ',
  'Codes': ['Importance of clinician relationships and interactions',
   'Parents pushing care/work up/attention forward',
   'Developmentally-appropriate ways to tell children']},
 {'Theme': 'Feeling that my child is safe',
  'Description': 'Their child’s safety was of great importance to parents and manifested in their stated fear of a cardiac event occurring and planning for that event. This also coincided with parents feeling as if they always had to be “on” and in some cases restricting a child’s activities even when it was not advised by clinicians.',
  'Codes': ['Planning for adverse events',
   'Parents always on',
   'Parent placed activity restrictions',
   'Fear of cardiac events',
   'Transition to adulthood']},
 {'Theme': 'Not feeling responsible for the diagnosis and its timing',
  'Description': 'Many parents reported distress around the concepts of what they could have done so the condition was identified earlier and the “what if’s” that could have happened if the condition were not found. This was expressed as guilt and remorse by parents that continued to persist long after the diagnosis was made.   ',
  'Codes': ['Why was it not found earlier',
   'Parental remorse',
   'What could have happened']},
 {'Theme': 'Appropriately coping with stress, anxiety and depression',
  'Description': 'Similar to individuals with a diagnosis of AAOCA, their parents also express their own stress, anxiety and depression that can result from their child’s AAOCA diagnosis. These were present regardless of their child’s age and independence. Again, similar to the individuals diagnosed with AAOCA, parents acknowledged that these mental health concerns will , they sought to be able to appropriately cope with them and reduce the impact and interference in their daily lives.',
  'Codes': ['Why was it not found earlier',
   'Parental remorse',
   'What could have happened',
   'Need for mental health support – parents',
   'Parents always “on”',
   'Fear of cardiac events',
   '']}]

In [ ]:
[i['Theme'] for i in human_themes]

In [ ]:
re_themes4=[{'name': 'Seeking reassurance through multiple medical opinions',
  'description': "Parents actively seek multiple medical opinions to gain reassurance and clarity about their child's condition. This pursuit reflects their need for confidence in treatment decisions and understanding of the diagnosis. They often feel overwhelmed by complex medical information and seek validation from various healthcare professionals to alleviate anxiety and uncertainty.",
  'codes': ['Seeking multiple opinions for reassurance and clarity',
   'Desire for positive medical opinions and reassurance']},
 {'name': 'Desiring comprehensive and statistical data on treatment outcomes',
  'description': "Parents express a strong desire for comprehensive data and statistics on treatment outcomes. They seek detailed information to make informed decisions about their child's care, reflecting a need for transparency and innovation in medical care. The lack of such data contributes to their frustration and anxiety.",
  'codes': ['Desire for comprehensive data on surgery outcomes for AAOCA',
   'Desire for advancements in surgical techniques and technology',
   'Frustration with lack of data and statistics']},
 {'name': 'Managing emotional challenges with proactive strategies',
  'description': "Parents experience significant emotional distress upon receiving their child's diagnosis and during treatment. They focus on specific actions to manage these feelings, such as seeking support, engaging in self-care, and staying informed. This proactive approach helps them cope with the uncertainty and emotional turmoil associated with their child's health condition.",
  'codes': ['Describing the emotional impact of diagnosis and treatment',
   'Emotional turmoil over surgical decisions',
   'The emotional toll of waiting for surgery outcomes',
   'Overwhelming feelings of uncertainty and fear for the future',
   "Anxiety and worry about child's health and future",
   'The emotional turmoil of receiving a life-threatening diagnosis']},
 {'name': "Balancing normalcy and protection in child's life",
  'description': 'Parents struggle with the desire for their children to lead active, normal lives while feeling the need to protect them from health risks. This internal conflict affects decisions about participation in sports and social activities, reflecting a wish for their children to experience typical childhood moments without the shadow of their health condition.',
  'codes': ["Desire for normalcy in the child's life despite health challenges",
   "Desire for normalcy in child's life post-surgery",
   "Fear of limitations on child's activities post-diagnosis",
   "Struggles with restrictions on child's activities"]},
 {'name': 'Living with constant vigilance and fear',
  'description': "Parents live in a state of heightened alertness, constantly monitoring their child's activities and health status due to the fear of sudden health crises. This vigilance is driven by the knowledge of potential risks associated with their child's condition, leading to anxiety about everyday activities.",
  'codes': ['Constant vigilance and fear of sudden health crises',
   "Constant vigilance and monitoring of child's activities",
   "Awareness of child's health status and emotional impact"]},
 {'name': 'Improving interactions with healthcare providers',
  'description': "Parents emphasize the importance of receiving emotional support and clear communication from healthcare professionals. They express frustration when medical explanations lack sensitivity, highlighting the need for compassionate care that addresses both the technical and emotional aspects of their child's condition.",
  'codes': ['Emotional support from medical professionals during crisis',
   'Desire for more emotional support from healthcare providers',
   'Desire for honest communication from medical professionals']},
 {'name': 'Experiencing relief and gratitude for successful interventions',
  'description': "Parents feel profound relief and gratitude when their child's medical interventions lead to positive outcomes. They recognize the importance of timely surgeries and effective treatments that allow their children to thrive. This relief is often accompanied by a celebration of their child's resilience and achievements, reflecting a hopeful outlook.",
  'codes': ['Relief and gratitude for successful medical interventions',
   'Joy in family time amidst health challenges',
   "Pride in child's resilience and achievements",
   'Relief from reassurance by medical professionals']},
 {'name': 'Struggling with guilt and self-blame',
  'description': "Parents often grapple with feelings of guilt and self-blame related to their child's health condition. They reflect on their role in their child's life and question their decisions, fearing they may have contributed to the child's health issues. This emotional struggle is intensified by comparisons with siblings.",
  'codes': ['Guilt over potential inaction and regret',
   "Guilt and self-blame for the child's condition",
   'Parental guilt over decision-making for their child']},
 {'name': 'Feeling overwhelmed by caregiving demands',
  'description': "Parents express feelings of being overwhelmed by the constant demands of caring for their children with AAOCA. Their lives revolve around their children's needs, leading to exhaustion and a loss of personal identity as they prioritize their children's health above their own.",
  'codes': ['Feeling overwhelmed by the demands of caring for children with AAOCA',
   "Emotional toll of living with a child's chronic condition"]},
 {'name': 'Building a supportive community with other parents',
  'description': "Parents actively seek support and community from others who have similar experiences. They turn to social media and support groups to share information, gain insights, and find comfort in knowing they are not alone in their struggles. This connection helps them manage the complexities of their child's condition.",
  'codes': ['Seeking support and community from other parents',
   'The importance of connecting with other parents in similar situations',
   'Desire for community support and shared experiences']},
 {'name': 'Advocating for awareness and understanding of heart conditions',
  'description': 'Parents feel a strong responsibility to raise awareness about heart conditions and the importance of early detection. They engage in advocacy efforts to educate others about the risks and necessary precautions for children with heart anomalies, reflecting their commitment to improving outcomes for others.',
  'codes': ['Advocacy for awareness of heart conditions',
   'Navigating medical uncertainty and trust']},
 {'name': 'Addressing frustration with the healthcare system',
  'description': 'Parents express frustration over the lack of awareness and understanding of AAOCA among healthcare professionals. They feel isolated in their experiences, often struggling to find clear information and support. This frustration is compounded by the emotional burden of dealing with a complex medical system.',
  'codes': ['Frustration with the lack of awareness among medical professionals',
   'Struggles with communication about the condition',
   'Challenges in discussing the condition with family and friends',
   'The struggle for information and understanding from medical professionals',
   'Frustration over missed early diagnosis of heart conditions']},
 {'name': "Long-term concerns about child's health",
  'description': "Parents express ongoing concerns about the long-term health monitoring required for their children with AAOCA, especially as they transition to adult care. They seek reassurance about their child's future health outcomes and express anxiety about potential complications, highlighting the need for more research and awareness.",
  'codes': ['Long-term health monitoring and transition to adult care',
   'Concerns about long-term outcomes and future health',
   'Need for more research on right coronary artery anomalies']}]

In [ ]:
[i['name'] for i in re_themes4]

In [ ]:
first_sentences_LLM = [i['description'].split('.')[0] for i in re_themes4]


In [ ]:
first_sentences_LLM_beofre = [i['description'].split('.')[0] for i in final_themes['major_themes']]


In [ ]:
first_sentences_human = [i['Description'].split('.')[0] for i in human_themes]


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer, util
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
import torch
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sentence_transformers import SentenceTransformer, util

# Load the model
model_name_eval = "all-MiniLM-L6-v2"
model = SentenceTransformer(model_name_eval)

# Encoding human and LLM descriptions to tensors
human_desc_embeddings = model.encode(first_sentences_human, convert_to_tensor=True)
llm_desc_embeddings = model.encode(first_sentences_LLM, convert_to_tensor=True)

# Ensure tensors are moved to CPU and converted to numpy
human_desc_embeddings = human_desc_embeddings.cpu()  # Move to CPU
llm_desc_embeddings = llm_desc_embeddings.cpu()  # Move to CPU

# Compute the cosine similarity matrix
cosine_sim_matrix = util.pytorch_cos_sim(human_desc_embeddings, llm_desc_embeddings).numpy()
rounded_cosine_sim_matrix = np.round(cosine_sim_matrix, 1)


# Create DataFrame to display the similarity matrix
similarity_desc_df = pd.DataFrame(
    rounded_cosine_sim_matrix,
    index=range(1,len([i['Theme'] for i in human_themes])+1),
    columns=range(1,len([i['name'] for i in re_themes4])+1)
)

# Plot the heatmap
plt.figure(figsize=(6, 5))
sns.heatmap(similarity_desc_df, annot=True, cmap="viridis", vmin=0, vmax=1, cbar_kws={'label': 'Similarity Score'})
plt.ylabel('Human Generated Themes',fontsize=14)
plt.xlabel('LLM Generated Themes After Evaluation',fontsize=14)
plt.show()


In [ ]:
import numpy as np

similarity_array = similarity_desc_df.to_numpy()
total_hit = np.sum(np.any(similarity_array >= .60, axis=1).astype(int))
hit_rate = total_hit/len(human_themes)
print(f"Hit Rate. {round(hit_rate,3)}")

In [ ]:
import torch
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sentence_transformers import SentenceTransformer, util

# Load the model
model_name_eval = "all-MiniLM-L6-v2"
model = SentenceTransformer(model_name_eval)

# Encoding human and LLM descriptions to tensors
human_desc_embeddings = model.encode(first_sentences_human, convert_to_tensor=True)
llm_desc_embeddings = model.encode(first_sentences_LLM_beofre, convert_to_tensor=True)

# Ensure tensors are moved to CPU and converted to numpy
human_desc_embeddings = human_desc_embeddings.cpu()  # Move to CPU
llm_desc_embeddings = llm_desc_embeddings.cpu()  # Move to CPU

# Compute the cosine similarity matrix
cosine_sim_matrix = util.pytorch_cos_sim(human_desc_embeddings, llm_desc_embeddings).numpy()
rounded_cosine_sim_matrix = np.round(cosine_sim_matrix, 1)


# Create DataFrame to display the similarity matrix
similarity_desc_df = pd.DataFrame(
    rounded_cosine_sim_matrix,
    index=range(1,len([i['Theme'] for i in human_themes])+1),
    columns=range(1,len([i['name'] for i in final_themes['major_themes']])+1)
)


# Plot the heatmap
plt.figure(figsize=(6, 5))
sns.heatmap(similarity_desc_df, annot=True, cmap="viridis", vmin=0, vmax=1, cbar_kws={'label': 'Similarity Score'})
plt.ylabel('Human Generated Themes',fontsize=14)
plt.xlabel('LLM Generated Themes Before Evaluation',fontsize=14)
plt.show()


In [ ]:
import numpy as np

similarity_array = similarity_desc_df.to_numpy()
total_hit = np.sum(np.any(similarity_array >= .60, axis=1).astype(int))
hit_rate = total_hit/len(human_themes)
print(f"Hit Rate. {round(hit_rate,3)}")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Data for the bars
labels = ['Before Evaluation', 'After Evaluation']
hit_rate = [0.83,0.92]  # Example hit rates

# Create a range of x positions for the bars with smaller spacing
x_pos = np.array([0.5, 0.6])  # Manually adjust the x positions to make them closer

# Create the plot with adjusted bar width and x positions
plt.figure(figsize=(6, 5))

# Bar plot
plt.bar(x_pos, hit_rate, color=['#1f77b4', '#2ca02c'], width=0.05, edgecolor='black')

# Add data labels on top of bars
# for i, v in enumerate(hit_rate):
#     plt.text(x_pos[i], v + 0.02, f'{v:.2f}', ha='center', fontsize=12, color='black')

# Adjust x-axis positions to make the bars closer
plt.xticks(x_pos, labels, fontsize=14)

# Add labels and title
plt.xlabel('LLM Generated Themes', fontsize=14)
plt.ylabel('Hit Rate in Human Generated Themes', fontsize=14)

# Customize ticks for readability
plt.tick_params(axis='both', which='major', labelsize=12)
plt.ylim([0.6,1])
# Remove gridlines
plt.grid(False)

# Show the plot
# plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer, util

# Initialize the model
model = SentenceTransformer("all-MiniLM-L6-v2")

# Assume 're_themes4' is a list of dictionaries with a key 'name'
llm_desc_embeddings = model.encode(first_sentences_human, convert_to_tensor=True)

# Compute the cosine similarity matrix
cosine_sim_matrix = util.pytorch_cos_sim(llm_desc_embeddings, llm_desc_embeddings)

# Convert to numpy array for easier handling
cosine_sim_matrix = cosine_sim_matrix.cpu().numpy()

# Create DataFrame to display the similarity matrix
similarity_desc_df = pd.DataFrame(
    cosine_sim_matrix,
    index=[i['Theme'] for i in human_themes],
    columns=[i['Theme'] for i in human_themes]
)

# Plot the heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(similarity_desc_df, annot=True, cmap="viridis", vmin=0, vmax=1, cbar_kws={'label': 'Similarity Score'})
plt.title('Similarity Matrix Heatmap of Themes')
plt.xlabel('Human Generated Themes')
plt.ylabel('Human Generated Themes')
plt.show()


In [ ]:
import numpy as np
print(f'{model_name_eval} Threshold: .50')
similarity_array = similarity_desc_df.to_numpy()
total_cells = similarity_array.size
high_similarity_count = np.sum(similarity_array >= .60)
percent_high = (high_similarity_count/total_cells)
print(f"Jarcard. {round(percent_high,3)}")

In [ ]:
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer, util

# Initialize the model
model = SentenceTransformer("all-MiniLM-L6-v2")

# Assume 're_themes4' is a list of dictionaries with a key 'name'
llm_desc_embeddings = model.encode(first_sentences_LLM, convert_to_tensor=True)

# Compute the cosine similarity matrix
cosine_sim_matrix = util.pytorch_cos_sim(llm_desc_embeddings, llm_desc_embeddings)

# Convert to numpy array for easier handling
cosine_sim_matrix = cosine_sim_matrix.cpu().numpy()

# Create DataFrame to display the similarity matrix
similarity_desc_df = pd.DataFrame(
    cosine_sim_matrix,
    index=[i['name'] for i in re_themes4],
    columns=[i['name'] for i in re_themes4]
)

# Plot the heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(similarity_desc_df, annot=True, cmap="viridis", vmin=0, vmax=1, cbar_kws={'label': 'Similarity Score'})
plt.title('Similarity Matrix Heatmap of Themes')
plt.xlabel('LLM Generated Themes after Evaluation')
plt.ylabel('LLM Generated Themes after Evaluation')
plt.show()


In [ ]:
import numpy as np
print(f'{model_name_eval} Threshold: .50')
similarity_array = similarity_desc_df.to_numpy()
total_cells = similarity_array.size
high_similarity_count = np.sum(similarity_array >= .60)
percent_high = (high_similarity_count/total_cells)
print(f"Jarcard. {round(percent_high,3)}")

In [ ]:
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer, util

# Initialize the model
model = SentenceTransformer("all-MiniLM-L6-v2")

# Assume 're_themes4' is a list of dictionaries with a key 'name'
llm_desc_embeddings = model.encode(first_sentences_LLM_beofre, convert_to_tensor=True)

# Compute the cosine similarity matrix
cosine_sim_matrix = util.pytorch_cos_sim(llm_desc_embeddings, llm_desc_embeddings)

# Convert to numpy array for easier handling
cosine_sim_matrix = cosine_sim_matrix.cpu().numpy()

# Create DataFrame to display the similarity matrix
similarity_desc_df = pd.DataFrame(
    cosine_sim_matrix,
    index=[i['name'] for i in final_themes['major_themes']],
    columns=[i['name'] for i in final_themes['major_themes']]
)

# Plot the heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(similarity_desc_df, annot=True, cmap="viridis", vmin=0, vmax=1, cbar_kws={'label': 'Similarity Score'})
plt.title('Similarity Matrix Heatmap of Themes')
plt.xlabel('LLM Generated Themes before Evaluation')
plt.ylabel('LLM Generated Themes before Evaluation')
plt.show()


In [ ]:
import numpy as np
print(f'{model_name_eval} Threshold: .50')
similarity_array = similarity_desc_df.to_numpy()
total_cells = similarity_array.size
high_similarity_count = np.sum(similarity_array >= .60)
percent_high = (high_similarity_count/total_cells)
print(f"Jarcard. {round(percent_high,3)}")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Data for Plot 1: Hit Rates
labels_1 = ['Before Evaluation', 'After Evaluation']
hit_rate = [0.83, 0.92]
x_pos_1 = np.arange(len(labels_1))

# Data for Plot 2: Jaccard-like Similarity
labels_2 = ['Before Evaluation', 'After Evaluation', 'Human Generated Themes']
jaccard_similarity = [0.29, 0.42, 0.33]
x_pos_2 = np.arange(len(labels_2))

# Create subplots
fig, axes = plt.subplots(1, 2, figsize=(12, 5))  # 1 row, 2 columns

# Plot 1 - Hit Rate
bars1 = axes[0].bar(x_pos_1, hit_rate, color=['#1f77b4', '#2ca02c'], width=0.6, edgecolor='black')
axes[0].set_xticks(x_pos_1)
axes[0].set_xticklabels(labels_1, fontsize=12)
axes[0].set_ylabel('Hit Rate in Human Themes', fontsize=12)
axes[0].set_ylim([0.6, 1])
axes[0].tick_params(axis='y', labelsize=10)
axes[0].set_title('LLM Hit Rate Comparison', fontsize=14)

# Add data labels
for i, v in enumerate(hit_rate):
    axes[0].text(x_pos_1[i], v + 0.015, f'{v:.2f}', ha='center', fontsize=11)

# Plot 2 - Jaccard-like Similarity
bars2 = axes[1].bar(x_pos_2, jaccard_similarity, color=['#1f77b4', '#2ca02c', '#ff7f0e'], width=0.6, edgecolor='black')
axes[1].set_xticks(x_pos_2)
axes[1].set_xticklabels(labels_2, fontsize=12, rotation=10)
axes[1].set_ylabel('Jaccard-like Similarity', fontsize=12)
axes[1].set_ylim([0, 0.5])
axes[1].tick_params(axis='y', labelsize=10)
axes[1].set_title('Theme Similarity Comparison', fontsize=14)

# Add data labels
for i, v in enumerate(jaccard_similarity):
    axes[1].text(x_pos_2[i], v + 0.015, f'{v:.2f}', ha='center', fontsize=11)

# Improve layout
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Data
hit_rate_labels = ['Before Evaluation', 'After Evaluation']
hit_rate = [0.83, 0.92]

similarity_labels = ['Before Evaluation', 'After Evaluation', 'Human Themes']
similarity = [0.29, 0.42, 0.33]

# Setup
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 6), facecolor='#f9f9f9')
fig.suptitle("Evaluation for LLM-Generated Themes", fontsize=18, weight='bold', color='#333')

# Custom color palettes
colors1 = ['#6baed6', '#31a354']
colors2 = ['#6baed6', '#31a354', '#fd8d3c']

# Plot 1 - Hit Rate
x1 = np.arange(len(hit_rate))
bars1 = ax1.bar(x1, hit_rate, color=colors1, edgecolor='black', width=0.5)

# Add value labels
for i, bar in enumerate(bars1):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.015, f"{hit_rate[i]*100:.0f}%", 
             ha='center', fontsize=12, color='#333')

# Style
# ax1.set_title("Hit Rate in Human Themes", fontsize=14, weight='bold')
ax1.set_xticks(x1)
ax1.set_xticklabels(hit_rate_labels, fontsize=12)
ax1.set_ylabel("Hit Rate (%)", fontsize=12)
ax1.set_ylim(0.6, 1.0)
ax1.set_facecolor("#ffffff")
ax1.spines[['top','right']].set_visible(False)

# Plot 2 - Jaccard-like Similarity
x2 = np.arange(len(similarity))
bars2 = ax2.bar(x2, similarity, color=colors2, edgecolor='black', width=0.5)

# Add value labels
for i, bar in enumerate(bars2):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.015, f"{similarity[i]*100:.0f}%", 
             ha='center', fontsize=12, color='#333')

# Style
# ax2.set_title("Theme Similarity (Jaccard-like)", fontsize=14, weight='bold')
ax2.set_xticks(x2)
ax2.set_xticklabels(similarity_labels, fontsize=12, rotation=10)
ax2.set_ylabel("Similarity (%)", fontsize=12)
ax2.set_ylim(0, 0.5)
ax2.set_facecolor("#ffffff")
ax2.spines[['top','right']].set_visible(False)

# Layout
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Data
hit_rate_labels = ['Before Evaluation', 'After Evaluation']
hit_rate = [0.83, 0.92]

similarity_labels = ['Before Evaluation', 'After Evaluation', 'Human Themes']
similarity = [0.42,0.29,0.33]

# Setup
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5), facecolor='#f9f9f9')

# Custom color palettes
colors1 = ['#6baed6', '#31a354']
colors2 = ['#6baed6', '#31a354', '#fd8d3c']

# Plot 1 - Hit Rate
x1 = np.arange(len(hit_rate))
bars1 = ax1.bar(x1, hit_rate, color=colors1, edgecolor='black', width=0.5)

# Add value labels
for i, bar in enumerate(bars1):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.015, f"{hit_rate[i]*100:.0f}%", 
             ha='center', fontsize=15, color='#333')

# Style
ax1.set_xticks(x1)
ax1.set_xticklabels(hit_rate_labels, fontsize=15)
ax1.set_ylabel("Hit Rate (%)", fontsize=15)
ax1.set_ylim(0.6, 1.0)
ax1.set_facecolor("#ffffff")
ax1.spines[['top','right']].set_visible(False)

# Plot 2 - Jaccard-like Similarity
x2 = np.arange(len(similarity))
bars2 = ax2.bar(x2, similarity, color=colors2, edgecolor='black', width=0.5)

# Add value labels
for i, bar in enumerate(bars2):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.015, f"{similarity[i]*100:.0f}%", 
             ha='center', fontsize=15, color='#333')

# Style
ax2.set_xticks(x2)
ax2.set_xticklabels(similarity_labels, fontsize=15, rotation=10)
ax2.set_ylabel("Within-Group Similarity (%)", fontsize=15)
ax2.set_ylim(0, 0.5)
ax2.set_facecolor("#ffffff")
ax2.spines[['top','right']].set_visible(False)

# Layout
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Data for the bars
labels = ['Before Evaluation', 'After Evaluation','Human Generated Themes']
hit_rate = [0.29, 0.42, 0.33]  # Example hit rates for the three categories

# Create a range of x positions for the bars
x_pos = np.array([0, 1, 2])  # Positions for the three bars

# Create the plot with adjusted bar width and x positions
plt.figure(figsize=(6, 5))

# Bar plot
bars = plt.bar(x_pos, hit_rate, color=['#1f77b4', '#2ca02c','#ff7f0e'], width=0.7, edgecolor='black')

# Add data labels on top of bars
# for i, v in enumerate(hit_rate):
#     plt.text(x_pos[i], v + 0.02, f'{v:.2f}', ha='center', fontsize=12, color='black')

# Adjust x-axis positions to make the bars closer
plt.xticks(x_pos, labels, fontsize=14,rotation=10)

# Add labels and title
# plt.xlabel('Evaluation Phases', fontsize=14)
plt.ylabel('Jaccard-like Similarity Percentage', fontsize=14)
# plt.title('Comparison of Hit Rates for Human Generated and Evaluated Themes', fontsize=16)

# Customize ticks for readability
plt.tick_params(axis='both', which='major', labelsize=12)

# Remove gridlines
plt.grid(False)

# Show the plot
plt.tight_layout()
plt.show()


##### similarity of theme names

In [ ]:
import torch
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sentence_transformers import SentenceTransformer, util

# Load the model
model_name_eval = "all-MiniLM-L6-v2"
model = SentenceTransformer(model_name_eval)

# Encoding human and LLM descriptions to tensors
human_desc_embeddings = model.encode([i['Theme'] for i in human_themes], convert_to_tensor=True)
llm_desc_embeddings = model.encode([i['name'] for i in re_themes4], convert_to_tensor=True)

# Ensure tensors are moved to CPU and converted to numpy
human_desc_embeddings = human_desc_embeddings.cpu()  # Move to CPU
llm_desc_embeddings = llm_desc_embeddings.cpu()  # Move to CPU

# Compute the cosine similarity matrix
cosine_sim_matrix = util.pytorch_cos_sim(human_desc_embeddings, llm_desc_embeddings).numpy()

# Create DataFrame to display the similarity matrix
similarity_desc_df = pd.DataFrame(
    cosine_sim_matrix,
    index=[i['Theme'] for i in human_themes],
    columns=[i['name'] for i in re_themes4]
)

# Plot the heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(similarity_desc_df, annot=True, cmap="viridis", vmin=0, vmax=1, cbar_kws={'label': 'Similarity Score'})
plt.title('Similarity Matrix Heatmap of Themes')
plt.ylabel('Human Generated Themes')
plt.xlabel('LLM Generated Themes after Evaluation')
plt.show()


In [ ]:
import numpy as np

similarity_array = similarity_desc_df.to_numpy()
total_hit = np.sum(np.any(similarity_array >= .40, axis=1).astype(int))
hit_rate = total_hit/len(human_themes)
print(f"Hit Rate. {round(hit_rate,3)}")

In [ ]:
import torch
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sentence_transformers import SentenceTransformer, util

# Load the model
model_name_eval = "all-MiniLM-L6-v2"
model = SentenceTransformer(model_name_eval)

# Encoding human and LLM descriptions to tensors
human_desc_embeddings = model.encode([i['Theme'] for i in human_themes], convert_to_tensor=True)
llm_desc_embeddings = model.encode([i['name'] for i in final_themes['major_themes']], convert_to_tensor=True)

# Ensure tensors are moved to CPU and converted to numpy
human_desc_embeddings = human_desc_embeddings.cpu()  # Move to CPU
llm_desc_embeddings = llm_desc_embeddings.cpu()  # Move to CPU

# Compute the cosine similarity matrix
cosine_sim_matrix = util.pytorch_cos_sim(human_desc_embeddings, llm_desc_embeddings).numpy()

# Create DataFrame to display the similarity matrix
similarity_desc_df = pd.DataFrame(
    cosine_sim_matrix,
    index=[i['Theme'] for i in human_themes],
    columns=[i['name'] for i in final_themes['major_themes']]
)

# Plot the heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(similarity_desc_df, annot=True, cmap="viridis", vmin=0, vmax=1, cbar_kws={'label': 'Similarity Score'})
plt.title('Similarity Matrix Heatmap of Themes')
plt.ylabel('Human Generated Themes')
plt.xlabel('LLM Generated Themes before Evaluation')
plt.show()


In [ ]:
similarity_array = similarity_desc_df.to_numpy()
total_hit = np.sum(np.any(similarity_array >= .40, axis=1).astype(int))
hit_rate = total_hit/len(human_themes)
print(f"Hit Rate. {round(hit_rate,3)}")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Data for the bars
labels = ['Before Evaluation', 'After Evaluation']
hit_rate = [0.58,0.67]  # Example hit rates

# Create a range of x positions for the bars with smaller spacing
x_pos = np.array([0, 0.6])  # Manually adjust the x positions to make them closer

# Create the plot with adjusted bar width and x positions
plt.figure(figsize=(7, 5))

# Bar plot
plt.bar(x_pos, hit_rate, color=['#1f77b4', '#2ca02c'], width=0.4, edgecolor='black')

# Add data labels on top of bars
# for i, v in enumerate(hit_rate):
#     plt.text(x_pos[i], v + 0.02, f'{v:.2f}', ha='center', fontsize=12, color='black')

# Adjust x-axis positions to make the bars closer
plt.xticks(x_pos, labels, fontsize=14)

# Add labels and title
plt.xlabel('LLM Generated Themes', fontsize=14)
plt.ylabel('Hit Rate in Human Generated Themes', fontsize=14)

# Customize ticks for readability
plt.tick_params(axis='both', which='major', labelsize=12)

# Remove gridlines
plt.grid(False)

# Show the plot
# plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer, util

# Initialize the model
model = SentenceTransformer("all-MiniLM-L6-v2")

# Assume 're_themes4' is a list of dictionaries with a key 'name'
llm_desc_embeddings = model.encode([i['Theme'] for i in human_themes], convert_to_tensor=True)

# Compute the cosine similarity matrix
cosine_sim_matrix = util.pytorch_cos_sim(llm_desc_embeddings, llm_desc_embeddings)

# Convert to numpy array for easier handling
cosine_sim_matrix = cosine_sim_matrix.cpu().numpy()

# Create DataFrame to display the similarity matrix
similarity_desc_df = pd.DataFrame(
    cosine_sim_matrix,
    index=[i['Theme'] for i in human_themes],
    columns=[i['Theme'] for i in human_themes]
)

# Plot the heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(similarity_desc_df, annot=True, cmap="viridis", vmin=0, vmax=1, cbar_kws={'label': 'Similarity Score'})
plt.title('Similarity Matrix Heatmap of Themes')
plt.xlabel('Human Generated Themes')
plt.ylabel('Human Generated Themes')
plt.show()


In [ ]:
import numpy as np
print(f'{model_name_eval} Threshold: .50')
similarity_array = similarity_desc_df.to_numpy()
total_cells = similarity_array.size
high_similarity_count = np.sum(similarity_array >= .40)
percent_high = (high_similarity_count/total_cells)
print(f"Jarcard. {round(percent_high,3)}")

In [ ]:
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer, util

# Initialize the model
model = SentenceTransformer("all-MiniLM-L6-v2")

# Assume 're_themes4' is a list of dictionaries with a key 'name'
llm_desc_embeddings = model.encode([i['name'] for i in re_themes4], convert_to_tensor=True)

# Compute the cosine similarity matrix
cosine_sim_matrix = util.pytorch_cos_sim(llm_desc_embeddings, llm_desc_embeddings)

# Convert to numpy array for easier handling
cosine_sim_matrix = cosine_sim_matrix.cpu().numpy()

# Create DataFrame to display the similarity matrix
similarity_desc_df = pd.DataFrame(
    cosine_sim_matrix,
    index=[i['name'] for i in re_themes4],
    columns=[i['name'] for i in re_themes4]
)

# Plot the heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(similarity_desc_df, annot=True, cmap="viridis", vmin=0, vmax=1, cbar_kws={'label': 'Similarity Score'})
plt.title('Similarity Matrix Heatmap of Themes')
plt.xlabel('LLM Generated Themes after Evaluation')
plt.ylabel('LLM Generated Themes after Evaluation')
plt.show()


In [ ]:
import numpy as np
print(f'{model_name_eval} Threshold: .50')
similarity_array = similarity_desc_df.to_numpy()
total_cells = similarity_array.size
high_similarity_count = np.sum(similarity_array >= .40)
percent_high = (high_similarity_count/total_cells)
print(f"Jarcard. {round(percent_high,3)}")

In [ ]:
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer, util

# Initialize the model
model = SentenceTransformer("all-MiniLM-L6-v2")

# Assume 're_themes4' is a list of dictionaries with a key 'name'
llm_desc_embeddings = model.encode([i['name'] for i in final_themes['major_themes']], convert_to_tensor=True)

# Compute the cosine similarity matrix
cosine_sim_matrix = util.pytorch_cos_sim(llm_desc_embeddings, llm_desc_embeddings)

# Convert to numpy array for easier handling
cosine_sim_matrix = cosine_sim_matrix.cpu().numpy()

# Create DataFrame to display the similarity matrix
similarity_desc_df = pd.DataFrame(
    cosine_sim_matrix,
    index=[i['name'] for i in final_themes['major_themes']],
    columns=[i['name'] for i in final_themes['major_themes']]
)

# Plot the heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(similarity_desc_df, annot=True, cmap="viridis", vmin=0, vmax=1, cbar_kws={'label': 'Similarity Score'})
plt.title('Similarity Matrix Heatmap of Themes')
plt.xlabel('LLM Generated Themes before Evaluation')
plt.ylabel('LLM Generated Themes before Evaluation')
plt.show()


In [ ]:
import numpy as np
print(f'{model_name_eval} Threshold: .50')
similarity_array = similarity_desc_df.to_numpy()
total_cells = similarity_array.size
high_similarity_count = np.sum(similarity_array >= .40)
percent_high = (high_similarity_count/total_cells)
print(f"Jarcard. {round(percent_high,3)}")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Data for the bars
labels = ['Human Generated Themes', 'Before Evaluation', 'After Evaluation']
hit_rate = [0.18, 0.35, 0.14]  # Example hit rates for the three categories

# Create a range of x positions for the bars
x_pos = np.array([0, 1, 2])  # Positions for the three bars

# Create the plot with adjusted bar width and x positions
plt.figure(figsize=(7, 5))

# Bar plot
bars = plt.bar(x_pos, hit_rate, color=['#ff7f0e', '#1f77b4', '#2ca02c'], width=0.4, edgecolor='black')

# Add data labels on top of bars
# for i, v in enumerate(hit_rate):
#     plt.text(x_pos[i], v + 0.02, f'{v:.2f}', ha='center', fontsize=12, color='black')

# Adjust x-axis positions to make the bars closer
plt.xticks(x_pos, labels, fontsize=14)

# Add labels and title
plt.xlabel('Evaluation Phases', fontsize=14)
plt.ylabel('Jaccard-like Similarity Percentage', fontsize=14)
# plt.title('Comparison of Hit Rates for Human Generated and Evaluated Themes', fontsize=16)

# Customize ticks for readability
plt.tick_params(axis='both', which='major', labelsize=12)

# Remove gridlines
plt.grid(False)

# Show the plot
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import pandas as pd

# Assuming similarity_desc_df is your DataFrame, and you've already converted it to a numpy array
similarity_array = similarity_desc_df.to_numpy()

# Set the threshold value (0.50 in this case)
threshold = 0.50

# Total number of cells in the similarity matrix (excluding diagonal)
total_cells = similarity_array.size - similarity_array.shape[0]  # Exclude diagonal cells

# Initialize counters
high_similarity_count = np.sum(similarity_array >= threshold)  # Count similarities above threshold
distinct_pairs = 0  # To count how many pairs are distinct

# Iterate over all pairs (i < j) to check similarity
for i in range(similarity_array.shape[0]):
    for j in range(i + 1, similarity_array.shape[1]):
        if similarity_array[i, j] < threshold:
            distinct_pairs += 1  # If similarity is below the threshold, the pair is distinct

# Calculate the percentage of high similarity and distinct pairs
percent_high = (high_similarity_count / total_cells)  # Percentage of high similarity pairs
percent_distinct = (distinct_pairs / total_cells)  # Percentage of distinct pairs

# Print the results
print(f"Threshold: {threshold}")
print(f"Jaccard-like similarity percentage: {round(percent_high, 3)}")  # Percentage of pairs above threshold
print(f"Distinct pairs percentage: {round(percent_distinct, 3)}")  # Percentage of distinct pairs
